# Word sense disambiguation with Simplified Lesk for Ukrainian

## Senses

Send requests to http://sum.in.ua/ to collect senses for a word.

In [1]:
from lxml import html
import requests
import numpy as np
import math

In [2]:
# http://sum.in.ua/ uses custom transliteration

conversion = {
    u'\u0410' : 'A',    u'\u0430' : 'a',
    u'\u0411' : 'B',    u'\u0431' : 'b',
    u'\u0412' : 'V',    u'\u0432' : 'v',
    u'\u0413' : 'Gh',    u'\u0433' : 'gh',
    u'\u0490' : 'G',    u'\u0491' : 'g',
    u'\u0414' : 'D',    u'\u0434' : 'd',
    u'\u0415' : 'E',    u'\u0435' : 'e',
    u'\u0404' : 'Ye',   u'\u0454' : 'je',
    u'\u0416' : 'Zh',   u'\u0436' : 'zh',
    u'\u0417' : 'Z',    u'\u0437' : 'z',
    u'\u0418' : 'Y',    u'\u0438' : 'y',
    u'\u0406' : 'I',    u'\u0456' : 'i',
    u'\u0407' : 'Ji',   u'\u0457' : 'ji',
    u'\u0419' : 'J',    u'\u0439' : 'j',
    u'\u041a' : 'K',    u'\u043a' : 'k',
    u'\u041b' : 'L',    u'\u043b' : 'l',
    u'\u041c' : 'M',    u'\u043c' : 'm',
    u'\u041d' : 'N',    u'\u043d' : 'n',
    u'\u041e' : 'O',    u'\u043e' : 'o',
    u'\u041f' : 'P',    u'\u043f' : 'p',
    u'\u0420' : 'R',    u'\u0440' : 'r',
    u'\u0421' : 'S',    u'\u0441' : 's',
    u'\u0422' : 'T',    u'\u0442' : 't',
    u'\u0423' : 'U',    u'\u0443' : 'u',
    u'\u0424' : 'F',    u'\u0444' : 'f',
    u'\u0425' : 'Kh',    u'\u0445' : 'kh',
    u'\u0426' : 'C',   u'\u0446' : 'c',
    u'\u0427' : 'Ch',   u'\u0447' : 'ch',
    u'\u0428' : 'Sh',   u'\u0448' : 'sh',
    u'\u0429' : 'Shh',  u'\u0449' : 'shh',
    u'\u044c' : 'j',
    u'\u042e' : 'Ju',   u'\u044e' : 'ju',
    u'\u042f' : 'Ja',   u'\u044f' : 'ja',
    u'\'' : '.'
}

def transliterate(word):
    translit = []
    for c in word:
        translit.append(conversion.get(c, c))
    return ''.join(translit)

In [3]:
print(transliterate("серце"))
print(transliterate("знущається"))
print(transliterate("бур'ян"))

serce
znushhajetjsja
bur.jan


In [4]:
URL = "http://sum.in.ua/s/"

def get_senses(word):
    """
    Extract senses for WORD from the dictionary.
    """
    word = transliterate(word)
    page = requests.get(URL + word)
    tree = html.fromstring(page.text)
    senses = tree.xpath("//p[@class='znach']")
    print(len(senses), "senses found.")
    return list(enumerate([" ".join(i.xpath('text()')) for i in senses], 1))

In [5]:
word = "лист"
senses = get_senses(word)
print(senses[2][1])

5 senses found.
 Тонкий, щільний шматок або шар
якого-небудь матеріалу (паперу, заліза, фанери і т. ін.).
Газорізальник Мамайчук-старший.. щось викроює з
товстого сталевого листа  ;
Цілий вечір Мишуня казився й дурів.. Зачепив неумисно
й перекинув відро з водою. Упустив фанерного листа,
на якому лежало тісто  ;
Шиферний лист іде на покрівлю житлових і промислових
будівель  .



## Collect sentenses from the brown-uk corpus

Read files from https://github.com/brown-uk/corpus/tree/master/data/good. Find all sentences that contain the word in question.

Use https://github.com/lang-uk/tokenize-uk for tokenization:
pip install tokenize_uk

and https://github.com/kmike/pymorphy2 for lemmatization:

pip install -U https://github.com/kmike/pymorphy2/archive/master.zip#egg=pymorphy2

pip install -U pymorphy2-dicts-uk

In [6]:
import os
import re
import pymorphy2
import tokenize_uk

CORPUS_PATH = "data/good/"

morph = pymorphy2.MorphAnalyzer(lang='uk')

In [7]:
def find_sentences(text, word):
    """
    From TEXT select sentences that contain WORD.
    """
    sentences_w_word = []
    paragraphs = tokenize_uk.tokenize_text(text)
    for paragraph in paragraphs:
        for sentence in paragraph:
            for token in sentence:
                if morph.parse(token)[0].normal_form == word:
                    sentences_w_word.append(sentence)
                    break
    return sentences_w_word

In [8]:
data = []

for filename in os.listdir(CORPUS_PATH):
    with open(CORPUS_PATH + filename, "r") as f:
        text = f.read()
    body = text[text.find("<body>"):]
    body = re.sub(r"<[^<>]+>", "", body)
    data += find_sentences(body, word)

print(len(data))

94


In [9]:
data[0]

['До',
 'речі',
 ',',
 'тоді',
 'ж',
 'було',
 'вирішено',
 ',',
 'що',
 'після',
 'двотижневого',
 'перебування',
 'по',
 'домівках',
 'усі',
 'знову',
 'зберуться',
 'до',
 'козацького',
 'війська',
 ',',
 'щоб',
 'заслухати',
 '«',
 'реляцію',
 ',',
 'яку',
 'привезуть',
 'посли',
 '»',
 '.',
 '29',
 'червня',
 'в',
 'Чигирині',
 'знову',
 'збирається',
 '«',
 'чорна',
 '»',
 'рада',
 'для',
 'заслухання',
 'листа',
 'від',
 'А',
 '.',
 'Кисіля',
 'та',
 'підготовки',
 'відповіді',
 '.']

## Implement Simplified Lesk

### 1. Collect context words from senses

Tokenize, downcase, lemmatize, and remove stop-words.

In [14]:
def collect_bow(text):
    """
    Tokenize TEXT with tokenize_uk, lemmatize TEXT with pymorphy2,
    and remove functional words.
    """
    tokens = tokenize_uk.tokenize_words(text)
    bow = []
    for token in tokens:
        parse = morph.parse(token)[0]
        if parse.tag.POS not in {'NUMR', 'NPRO', 'PREP', 'CONJ', 'PRCL'} and \
            not 'PNCT' in parse.tag and \
            not token.isdigit() and \
            token not in ['до', 'на', 'за', 'його']:
            bow.append(parse.normal_form)
    return set(bow)

In [15]:
collect_bow(senses[0][1])

{'біла',
 'вертати',
 'вигляд',
 'виконати',
 'виноградний',
 'вирізуватися',
 'вусик',
 'відро',
 'газообмін',
 'глек',
 'головка',
 'диво',
 'дунай',
 'дівчина',
 'живлення',
 'звичайно',
 'згинути',
 'зелений',
 'зон',
 'капуста',
 'капустяний',
 'кращий',
 'краяти',
 'лист',
 'лівий',
 'літ',
 'маланчин',
 'марний',
 'метр',
 'молоко',
 'місячний',
 'на',
 'назад',
 'найти',
 'ніжно',
 'опадати',
 'орган',
 'падіння',
 'паляниця',
 'переворот',
 'петля',
 'плавно',
 'пластинка',
 'поволі',
 'повітряний',
 'право',
 'піти',
 'раз',
 'рок',
 'рослина',
 'ріка',
 'світа',
 'селянський',
 'сир',
 'складовий',
 'сонце',
 'сторонець',
 'стояти',
 'сяйво',
 'темрява',
 'терти',
 'тонкий',
 'тремтіти',
 'упливати',
 'частина',
 'чужий',
 'штольц'}

In [16]:
collect_bow(data[0])

{'бути',
 'вирішити',
 'відповідь',
 'військо',
 'двотижневий',
 'до',
 'домівка',
 'заслухання',
 'заслухати',
 'збиратися',
 'знову',
 'зібратися',
 'кисіль',
 'козацький',
 'лист',
 'перебування',
 'посол',
 'привезти',
 'підготовка',
 'після',
 'рад',
 'реляція',
 'річ',
 'червень',
 'чигирин',
 'чорна'}

### 2. Define the best sense for the word

Find the sense that has the biggest overlap with the word context. Use sense 1 as default.

In [17]:
signatures = [(idx, collect_bow(sense)) for idx, sense in senses]

def classify(sentence, word, senses, log=False):
    """
    Find the sense that has the biggest overlap with the word context.
    Use sense 1 as default.
    """
    context = collect_bow(sentence)
    best_sense = 1
    max_overlap = 0
    max_overlap_words = []
    for sense_id, signature in signatures:
        overlap = context.intersection(signature)
        
#         if score == max_overlap and sense_id == 4:
#             best_sense = sense_id
#             max_overlap_words = overlap
            
        
        if len(overlap) > max_overlap:
            max_overlap_words = overlap
            max_overlap = len(overlap)
            best_sense = sense_id
        
    if log:
        print(" ".join(sentence))
        print("Best sense:", best_sense)
        print("{} words overlap: {}.\n".format(max_overlap, ", ".join(max_overlap_words)))
    return best_sense

In [18]:
senses[1]

(2,
 '   у. Те саме, що  . — Я б прикрила твій\nслід листом, щоб його вітер не завіяв, піском не замів, —\nсказала Мелашка  ; Ах, яке ж\nсинє й зоряне небо! І запах цегли — руїн, жовтого\nлисту — аж у голову хмелем  ;\n \xa0 Засушене, рідше свіже листя деяких рослин, що\nвикористовується як приправа або для виготовлення напою\nчи ліків. Яких вже мені ліків не завдавали! Лукаш мене\nнапував листом от чорних порічок  ; Дівчата онде подались Збирати свіжий лист із чаю\n ; Не поспішаючи, він кладе\nв казанок.. сіль, дрібно покришену й підсмажену цибулю\nі лавровий лист  .')

In [19]:
classify(data[0], word, signatures, log=True)

До речі , тоді ж було вирішено , що після двотижневого перебування по домівках усі знову зберуться до козацького війська , щоб заслухати « реляцію , яку привезуть посли » . 29 червня в Чигирині знову збирається « чорна » рада для заслухання листа від А . Кисіля та підготовки відповіді .
Best sense: 4
3 words overlap: військо, після, лист.



4

In [20]:
classify(data[1], word, signatures, log=True)

Про ці обставини директор краєзнавчого музею неодноразово листами повідомляв голів Любомльської райдержадміністрації та районної ради , а також управління культури Волинської облдержадміністрації .
Best sense: 4
2 words overlap: лист, про.



4

In [21]:
classify(data[2], word, signatures, log=True)

Через кілька років народиться і другий цикл — « Кримські відгуки » , до якого увійдуть « Імпровізація » , « Уривки з листа » , « Мрії » , драматична поема « Іфігенія в Тавриді » , « Весна зимова » … Цей цикл — усе пережите Лесею в Ялті з червня 1897 до червня 1898 року …
Best sense: 1
2 words overlap: рок, лист.



1

In [22]:
classify(data[10], word, signatures, log=True)

Розказував Рей про свої шкільні роки і спонукав її до написання листа керівництву залізниці , адже це так важливо — отримати освіту .
Best sense: 4
2 words overlap: лист, про.



4

In [23]:
classify(data[50], word, signatures, log=True)

– складення та подання відкритих листів до влади та впливових осіб ;
Best sense: 4
2 words overlap: особа, лист.



4

In [24]:
classify(data[60], word, signatures, log=True)

Тимчасом приходить ще один лист - від музикантки , яку так давно люблю .
Best sense: 1
1 words overlap: лист.



1

In [25]:
from collections import Counter

sense_counts = []
for i in range(len(data)):
    sense_id = classify(data[i], word, signatures)
    sense_counts.append(sense_id)
Counter(sense_counts)

Counter({4: 28, 1: 52, 5: 2, 2: 7, 3: 5})

In [35]:
test_data = data[:50]

In [36]:
test_labels = [classify(x, word, signatures) for x in test_data]

In [37]:
senses

[(1,
  '   а. Орган повітряного живлення і\nгазообміну рослин у вигляді тонкої, звичайно зеленої\nпластинки. Лист за листом опадає, Рік за роком\nупливає. Назад не вертає!  ;\nВиноградний лист вирізується з темряви й ніжно тремтить\nразом з тонким вусиком у місячному сяйві  ; \xa0  А я піду на край світа... На\nчужій сторонці Найду кращу [дівчину] або згину, Як\nтой лист на сонці  ; Пішли літа\n[Маланчині] марне з світа, як лист по Дунаю...\n ;  \xa0 Складова частина головки\nкапусти. У відрі стоїть глек з молоком, сир у капустяному\nлисті, біла селянська паляниця  ;\n \xa0   листом,   — плавно, поволі.\n[Штольц:] В зоні виконати, три петлі, два праві й\nдва ліві перевороти.. Падіння листом до шестисот\nметрів  .\n    див.  .\n'),
 (2,
  '   у. Те саме, що  . — Я б прикрила твій\nслід листом, щоб його вітер не завіяв, піском не замів, —\nсказала Мелашка  ; Ах, яке ж\nсинє й зоряне небо! І запах цегли — руїн, жовтого\nлисту — аж у голову хмелем  ;\n \xa0 Засушене, рідше свіже листя деяк

In [38]:
test_set = list(zip(test_data, test_labels))

In [39]:
for idx, item in enumerate(test_set):
    print("Index:", idx)
    print("Sense:", item[1])
    print("Sample:", item[0])
    print("------------")

Index: 0
Sense: 4
Sample: ['До', 'речі', ',', 'тоді', 'ж', 'було', 'вирішено', ',', 'що', 'після', 'двотижневого', 'перебування', 'по', 'домівках', 'усі', 'знову', 'зберуться', 'до', 'козацького', 'війська', ',', 'щоб', 'заслухати', '«', 'реляцію', ',', 'яку', 'привезуть', 'посли', '»', '.', '29', 'червня', 'в', 'Чигирині', 'знову', 'збирається', '«', 'чорна', '»', 'рада', 'для', 'заслухання', 'листа', 'від', 'А', '.', 'Кисіля', 'та', 'підготовки', 'відповіді', '.']
------------
Index: 1
Sense: 4
Sample: ['Про', 'ці', 'обставини', 'директор', 'краєзнавчого', 'музею', 'неодноразово', 'листами', 'повідомляв', 'голів', 'Любомльської', 'райдержадміністрації', 'та', 'районної', 'ради', ',', 'а', 'також', 'управління', 'культури', 'Волинської', 'облдержадміністрації', '.']
------------
Index: 2
Sense: 1
Sample: ['Через', 'кілька', 'років', 'народиться', 'і', 'другий', 'цикл', '—', '«', 'Кримські', 'відгуки', '»', ',', 'до', 'якого', 'увійдуть', '«', 'Імпровізація', '»', ',', '«', 'Уривки', '

In [31]:
corrected_labels = test_labels.copy()
corrected_labels[2] = 4
corrected_labels[3] = 4
corrected_labels[5] = 4 
corrected_labels[6] = 5
corrected_labels[7] = 5
corrected_labels[9] = 4
corrected_labels[11] = 4
corrected_labels[12] = 4
corrected_labels[13] = 4
corrected_labels[14] = 4
corrected_labels[15] = 4
corrected_labels[16] = 4
corrected_labels[17] = 4
corrected_labels[18] = 4
corrected_labels[20] = 4
corrected_labels[23] = 4
corrected_labels[24] = 4
corrected_labels[25] = 2
corrected_labels[26] = 4
corrected_labels[29] = 4
corrected_labels[30] = 4
corrected_labels[31] = 4
corrected_labels[32] = 4
corrected_labels[36] = 5
corrected_labels[38] = 5
corrected_labels[39] = 1
corrected_labels[40] = 4
corrected_labels[41] = 5
corrected_labels[42] = 4
corrected_labels[43] = 1
corrected_labels[45] = 4
corrected_labels[47] = 4
corrected_labels[49] = 4

In [42]:
score = np.sum(np.array(test_labels) == np.array(corrected_labels)) / len(corrected_labels)

In [43]:
print(f"Baseline quality of Lesk {score * 100:.2f}%")

Baseline quality of Lesk 34.00%


## TF-IDF weighting

In [56]:
signatures = [(idx, collect_bow(sense)) for idx, sense in senses]

In [46]:
signature_words = {w for s in signatures for w in s[1]}

In [47]:
signature_words_counts = []
for word in signature_words:
    for sentence in test_data:
        if word in collect_bow(sentence):
            signature_words_counts.append(word)

In [48]:
docs_count = 50
signature_counter = Counter(signature_words_counts)

In [79]:

def classify(sentence, word, senses, log=False):
    """
    Find the sense that has the biggest overlap with the word context.
    Use sense 1 as default.
    """
    context = collect_bow(sentence)
    best_sense = 1
    max_overlap = 0
    max_overlap_words = []
    for sense_id, signature in signatures:
        overlap = context.intersection(signature)

        score = np.array([math.log(50, 2 + signature_counter[x]) for x in overlap]).sum()
#         if score == max_overlap and sense_id == 4:
#             best_sense = sense_id
#             max_overlap_words = overlap
            
        if score > max_overlap:
            max_overlap_words = overlap
            max_overlap = score
            best_sense = sense_id
            
       
    if log:
        print(" ".join(sentence))
        print("Best sense:", best_sense)
        print("{} words overlap: {}.\n".format(max_overlap, ", ".join(max_overlap_words)))
    return best_sense

In [80]:
predicted = [classify(x, word, signatures, log=False) for x in test_data]

In [81]:
score = np.sum(np.array(predicted) == np.array(corrected_labels)) / len(corrected_labels)

In [82]:
print(f"Improved quality of Lesk {score * 100:.2f}%")

Improved quality of Lesk 34.00%


## Extend the word list

In [92]:
senses

[(1,
  '   а. Орган повітряного живлення і\nгазообміну рослин у вигляді тонкої, звичайно зеленої\nпластинки. Лист за листом опадає, Рік за роком\nупливає. Назад не вертає!  ;\nВиноградний лист вирізується з темряви й ніжно тремтить\nразом з тонким вусиком у місячному сяйві  ; \xa0  А я піду на край світа... На\nчужій сторонці Найду кращу [дівчину] або згину, Як\nтой лист на сонці  ; Пішли літа\n[Маланчині] марне з світа, як лист по Дунаю...\n ;  \xa0 Складова частина головки\nкапусти. У відрі стоїть глек з молоком, сир у капустяному\nлисті, біла селянська паляниця  ;\n \xa0   листом,   — плавно, поволі.\n[Штольц:] В зоні виконати, три петлі, два праві й\nдва ліві перевороти.. Падіння листом до шестисот\nметрів  .\n    див.  .\n'),
 (2,
  '   у. Те саме, що  . — Я б прикрила твій\nслід листом, щоб його вітер не завіяв, піском не замів, —\nсказала Мелашка  ; Ах, яке ж\nсинє й зоряне небо! І запах цегли — руїн, жовтого\nлисту — аж у голову хмелем  ;\n \xa0 Засушене, рідше свіже листя деяк

In [179]:
enrichment = {
    1: "", 
    2: "",
    3: "",
    4: "У', 'листі', 'до', 'приятельки', 'Софія', 'Шептицька', 'написала', ',', 'що', 'це', 'хлоп’я' 'Але', 'якщо', 'підписок', 'буде', 'дуже', 'багато', ',', 'Твоя', 'електронна', 'скринька', 'може', 'вщерть', 'заповнитися' 'У', 'зв’язку', 'з', 'малюнками', 'до', 'останнього', ',', 'А', '.', 'Ждаха', 'згадується', ',', 'зокрема', ',', 'в', 'листах', 'до', 'випускника',  'Відповідно', 'до', 'листа', 'МОН', 'України', 'щодо', 'викладання', 'економіки', 'Ображена', 'на', 'тебе', 'за', 'зради', ',', 'ображена', 'на', 'себе', На підставі цих даних міський голова надіслав 'мешканці', 'селища', 'підписали' 'коли', 'закрили', 'станцію',", 
    5: ""
}

In [180]:
senses_rich = []
for sid, stext in senses:
    senses_rich.append((sid, stext + enrichment[sid]))

In [181]:
signatures = [(idx, collect_bow(sense)) for idx, sense in senses_rich]

In [182]:
predicted = [classify(x, word, signatures, log=False) for x in test_data]

In [183]:
score = np.sum(np.array(predicted) == np.array(corrected_labels)) / len(corrected_labels)

In [184]:
print(f"Improved quality of Lesk {score * 100:.2f}%")

Improved quality of Lesk 66.00%


### Adjust classifier to corpus

In [185]:

def classify(sentence, word, senses, log=False):
    """
    Find the sense that has the biggest overlap with the word context.
    Use sense 1 as default.
    """
    context = collect_bow(sentence)
    best_sense = 1
    max_overlap = 0
    max_overlap_words = []
    for sense_id, signature in signatures:
        overlap = context.intersection(signature)

        score = np.array([math.log(50, 2 + signature_counter[x]) for x in overlap]).sum()
        if score == max_overlap and sense_id == 4:
            best_sense = sense_id
            max_overlap_words = overlap
            
        if score > max_overlap:
            max_overlap_words = overlap
            max_overlap = score
            best_sense = sense_id
            
       
    if log:
        print(" ".join(sentence))
        print("Best sense:", best_sense)
        print("{} words overlap: {}.\n".format(max_overlap, ", ".join(max_overlap_words)))
    return best_sense

In [186]:
predicted = [classify(x, word, signatures, log=False) for x in test_data]

In [187]:
score = np.sum(np.array(predicted) == np.array(corrected_labels)) / len(corrected_labels)

In [188]:
print(f"Improved quality of Lesk {score * 100:.2f}%")

Improved quality of Lesk 74.00%


## Your task

1. Choose a word (or a list of words).
2. Collect real-world sentences for the word(s).
3. Implement Simplified Lesk for disambiguating the word(s).
4. Set aside 50 sentences as a test set and manually fix the disambiguation errors that Lesk made. Calculate the quality of WSD by Lesk.
5. Improve the quality of WSD by:
- taking into account word count
- counting IDF score (log of total number of senses divided by the number of senses where the word was used)
- extending the word lists for senses in a semi-supervised manner (using the context of words that were disambiguated with high confidence)